# GOES Infrared Satellite Images
### Channels avaliable in this notebook are:
* 13 - Clean Longwave IR:  10.3 $\mu m$

## Justin Richling
## 09/20/18

http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/goes16_download.cgi

## Imports

In [1]:
# Random Library Imports
import subprocess,os,glob

# Importing Datetime Libraries
from datetime import datetime, date

# CartoPy Map Plotting Libraires
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Numerical and Scientific Libraries
import numpy as np

# Accessing Data from External Databases via XLM Catalog
from siphon.ncss import NCSS
from siphon.catalog import TDSCatalog

# MetPy Libraries
import metpy
import metpy.calc as mpcalc
from metpy.units import units
from metpy.plots import ctables
from metpy.plots import add_metpy_logo
from metpy.plots.ctables import registry
from metpy.plots import add_timestamp

# NetCDF Libraries
from netCDF4 import Dataset
from netCDF4 import num2date

# Matplotlib Plotting Libraries
#import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm, Normalize
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import patheffects



In [2]:
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeat

In [2]:
# Pull the current time
now = datetime.now()

In [3]:
# Set the font 
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 18,
        }

In [4]:
# Unidata/MetPy Dictionary of GOES channels

channel_list = {u'1 - Blue Band 0.47 \u03BCm': 1,
                u'2 - Red Band 0.64 \u03BCm': 2,
                u'3 - Veggie Band 0.86 \u03BCm': 3,
                u'4 - Cirrus Band 1.37 \u03BCm': 4,
                u'5 - Snow/Ice Band 1.6 \u03BCm': 5,
                u'6 - Cloud Particle Size Band 2.2 \u03BCm': 6,
                u'7 - Shortwave Window Band 3.9 \u03BCm': 7,
                u'8 - Upper-Level Tropo. WV Band 6.2 \u03BCm': 8,
                u'9 - Mid-Level Tropo. WV Band 6.9 \u03BCm': 9,
                u'10 - Low-Level WV Band 7.3 \u03BCm': 10,
                u'11 - Cloud-Top Phase Band 8.4 \u03BCm': 11,
                u'12 - Ozone Band 9.6 \u03BCm': 12,
                u'13 - Clean IR Longwave Band 10.3 \u03BCm': 13,
                u'14 - IR Longwave Band 11.2 \u03BCm': 14,
                u'15 - Dirty Longwave Band 12.3 \u03BCm': 15,
                u'16 - CO2 Longwave IR 13.3 \u03BCm': 16}

# Archived Data

In [51]:
# Set the date you want to convert
dt = datetime(2019, 8, 25)
#dt = now

# Start of year for reference
d0 = datetime(2019, 1, 1)

# Find the difference and add one to get the day number of the calander year
delta = dt - d0
Julian_Day = delta.days+1
if Julian_Day < 100:
    Julian_Day = "0"+str(Julian_Day)
    if int(Julian_Day) < 10:
        Julian_Day = "0"+str(Julian_Day)

Year = str('{0:%Y}'.format(dt))
Month = str('{0:%m}'.format(dt))
Day = str('{0:%d}'.format(dt))
Hour = str('{0:%H}'.format(dt))
Minute = str('{0:%m}'.format(dt))

#'{0:%Y}'.format(dt)+"-"+'{0:%m}'.format(dt)+"-"+'{0:%d}'.format(dt)+"-"+'{0:%H}'.format(dt)
print("date: ",Year+"-"+Month+"-"+Day+"-"+Hour+"-"+Minute)

# Julian day (Day)
print("Julian number: ",Julian_Day)

date:  2019-08-25-00-08
Julian number:  237


In [35]:
%%time

# rclone AWS data access via remote server
# Day must be Julian day number
! rclone --include "OR_*-M6C13*.nc" copy Public-AWS:noaa-goes16/ABI-L2-CMIPC/2020/130/00/ ./


CPU times: user 306 ms, sys: 102 ms, total: 408 ms
Wall time: 14.9 s


In [36]:
# Check to make sure the data went to the right path
GOES_sample_path = "/Users/chowdahead/Downloads/WX_Data/GOES_Data"
GOES_sample_path

'/Users/chowdahead/Downloads/WX_Data/GOES_Data'

# --------------------------------------------------------------------------------------------

# Populate the Lists for Different Channels

In [37]:
ls

Airmass/
Dust/
GOES_rgb_airmass.png
GOES_rgb_convection.png
GOES_rgb_true_color.png
OR_ABI-L1b-RadC-M6C01_G16_s20201391801120_e20201391803493_c20201391803545.nc
OR_ABI-L1b-RadC-M6C02_G16_s20201391801120_e20201391803493_c20201391803522.nc
OR_ABI-L1b-RadC-M6C03_G16_s20201391801120_e20201391803493_c20201391804023.nc
OR_ABI-L1b-RadC-M6C05_G16_s20201391801120_e20201391803493_c20201391803585.nc
OR_ABI-L1b-RadC-M6C07_G16_s20201391801120_e20201391803505_c20201391803557.nc
OR_ABI-L1b-RadC-M6C08_G16_s20201391801120_e20201391803493_c20201391803599.nc
OR_ABI-L1b-RadC-M6C10_G16_s20201391801120_e20201391803504_c20201391803579.nc
OR_ABI-L1b-RadC-M6C11_G16_s20201391801120_e20201391803493_c20201391803570.nc
OR_ABI-L1b-RadC-M6C12_G16_s20201391801120_e20201391803498_c20201391803551.nc
OR_ABI-L1b-RadC-M6C13_G16_s20201391801120_e20201391803504_c20201391804013.nc
OR_ABI-L1b-RadC-M6C14_G16_s20201391801120_e20201391803493_c20201391803590.nc
OR_ABI-L1b-RadC-M6C15_G16_s20201391801120_e20201391803499_c2020139180

In [51]:
GOES_samples = []

for name in glob.glob(GOES_sample_path+'/*CMIPC*M6C13*.nc'):
    GOES_samples.append(name)
    #for name in glob.glob(GOES_sample_path+'OR*L2*RadC*C09*G16*s'+Year+str(Julian_Day)+'*.nc'):
    #    GOES16_samples_9.append(name)

GOES_samples = sorted(GOES_samples)
print(len(GOES_samples))
print(GOES_samples[0])

12
/Users/chowdahead/Downloads/WX_Data/GOES_Data/OR_ABI-L2-CMIPC-M6C13_G16_s20201300001113_e20201300003497_c20201300004017.nc


# Single images here; loops are further down in the notebook

### Set the lat/lon extent of the map

In [47]:
# Lon/Lat Box

# [Lon_0, Lon_1, Lat_0, Lat_1]
# CONUS
extent = [-120., -70, 20., 50.] 
#extent = [-102.97, -96.97, 35.75, 39.75]

## IR - Channel 13
With the help of searching through some Unidata public emails, I was able to find the color table for the NESDIS IR colorbar. 
* https://www.unidata.ucar.edu/mailing_lists/archives/ldm-users/2018/msg00055.html

In [12]:
cd ~/

/Users/chowdahead


In [40]:
import GOES_IR_ColorMap as IR

For plotting (ie pcolormesh, imshow, etc.) set the max/min:
vmin=162.  vmax=330.


<h1>We can also define a map projection function that can take details of the different channels</h1>

* path - data file location
* channel - which GOES channel number
* title - plot title
* savepath - image save location
* my_cmap - (optional) desired color table
* vmin - (optional) min value for plotting
* vmax - (optional) max value for plotting

In [49]:
def Map(path,channel,savepath,extent,my_cmap=None,vmin=None,vmax=None):
    vis_name = path[-76:]
    title = "GOES-"+vis_name[23:25]+" Ch "+str(channel)+" - IR"
    nc = Dataset(path)
    data = nc.variables['CMI'][:]
            #ch2nc.close()
            #ch2nc = None
        
    sat_h = nc.variables['goes_imager_projection'].perspective_point_height
    
    X = nc.variables['x'][:] * sat_h
    Y = nc.variables['y'][:] * sat_h

    proj_var = nc.variables['goes_imager_projection']
    sat_height = proj_var.perspective_point_height
    central_lon = proj_var.longitude_of_projection_origin
    semi_major = proj_var.semi_major_axis
    semi_minor = proj_var.semi_minor_axis

# Create new figure
    #fig = plt.figure(figsize=(17., 11.))
    fig = plt.figure(figsize=(17,11))

# Add state boundaries to plot
    states_boundaries = cfeature.NaturalEarthFeature(category='cultural',
        name='admin_1_states_provinces_lakes',scale='50m', facecolor='none')

    country_borders = cfeature.NaturalEarthFeature(category='cultural',
        name='admin_0_countries',scale='50m', facecolor='none')
    
# Set Projection of Plot
    globe = ccrs.Globe(semimajor_axis=semi_major, semiminor_axis=semi_minor)
    crs = ccrs.Geostationary(central_longitude=central_lon, satellite_height=sat_height, globe=globe)
    #plotcrs = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)
    plotcrs = ccrs.LambertConformal(central_longitude=-99.969 ,central_latitude=37.761)

# Add the map and set the extent
    ax = plt.subplot(111, projection=plotcrs) 
        
    #reader = shpreader.Reader('/Users/chowdahead/Documents/shapefiles/countyl010g_shp_nt00964/countyl010g.shp')
    #counties = list(reader.geometries())
    #COUNTIES = cfeat.ShapelyFeature(counties,ccrs.PlateCarree())
    #ax.add_feature(COUNTIES, facecolor='none',edgecolor='r')
# Find and convert Julian day to date    
    import datetime as DT
    dt = datetime(2019,1,1)
    #print(vis_name[31:34])
    dtdelta = DT.timedelta(days=int(vis_name[31:34])-1)
    Day = dt + dtdelta
    im_file = vis_name[27:31]+"_"+'{0:%m_%d}'.format(Day)+"_"+vis_name[34:36]+vis_name[36:38]
    title_date = vis_name[27:31]+" "+'{0:%B %d}'.format(Day)+" "+vis_name[34:36]+""+vis_name[36:38]+"Z"
    im_file = vis_name[27:31]+"_"+'{0:%m_%d}'.format(Day)+"_"+vis_name[34:36]+vis_name[36:38]

# Set the plot title    
    #plt.title(title,loc='left',fontdict=font)
    #plt.title(title_date,loc='right',fontdict=font)
    
    text_time = ax.text(.995, 0.01, 
        vis_name[27:31]+" "+'{0:%B %d}'.format(Day)+" "+vis_name[34:36]+""+vis_name[36:38]+"Z",
        horizontalalignment='right', transform=ax.transAxes,
        color='white', fontsize=20, weight='bold')

    text_time2 = ax.text(0.005, 0.01, 
        title,
        horizontalalignment='left', transform=ax.transAxes,
        color='white', fontsize=20, weight='bold')

    outline_effect = [patheffects.withStroke(linewidth=5, foreground='black')]
    text_time.set_path_effects(outline_effect)
    text_time2.set_path_effects(outline_effect)

# Add state boundaries to plot
    ax.add_feature(states_boundaries, edgecolor='blue', linewidth=1)

# Add country borders to plot
    ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Set the plotting extent    
    #ax.set_extent(extent, ccrs.PlateCarree()) # Eastern Pacific

# Plot the image
    #img = ax.imshow(data[:],origin='upper',vmin=vmin,vmax=vmax,extent=(X.min(), X.max(), Y.min(), Y.max()),
    #    interpolation='nearest',transform=crs,cmap=my_cmap) 

    ax.set_extent(extent)

    ax.gridlines(color="w", linestyle="dotted",alpha=0.5)

    im = plt.imshow(data[:],origin='upper',extent=(X.min(), X.max(), Y.min(), Y.max()),
                interpolation='nearest',vmin=162.,vmax=330.,cmap=my_cmap, transform=crs) #,vmin=vmin,vmax=vmax
    cbar = plt.colorbar(im, orientation='vertical',pad=0.005,aspect=50)
    
# Display the figure
    #plt.show()
    
# Set the name for saved figure, and save it 

    
    #day_path = savepath+'{0:%m_%d}'.format(Day)+"/"
    #if not os.path.isdir(day_path):
    #    os.makedirs(day_path)
    #print(day_path)
    #fig.savefig(outfile,bbox_inches='tight',dpi=120)
    outfile = "GOES"+vis_name[23:25]+"_Ch"+str(channel)+"_"+str(im_file)+".png"
    fig.savefig(save_path+outfile,bbox_inches='tight')#,dpi=120
    plt.close(fig)

In [43]:
save_path = "/Users/chowdahead/Desktop/"

## We can now imput data from the different channels and have our generic map maker function run on each to produce our satellite images!

### Map Arguments: Map(path, channel, title, savepath, my_cmap, vmin=None, vmax=None)

In [ ]:
save_path

In [50]:
# Map(path,channel,title,savepath,extent,my_cmap=None,vmin=None,vmax=None)

i = 2
channel=13
#Map(GOES16_samples_13[i],channel,title,save_path,extent,IR_cmap,vmin=162.,vmax=330.)
Map(GOES16_samples_13[i],channel,save_path,extent,my_cmap=IR.IR_Colormap())

<h2>----------------------------------------------//---------------------------------------------------------</h2>
<h2>----------------------------------------------//---------------------------------------------------------</h2>

<h1>We can also take the lists of files we created earlier to plot multiple timesteps and create animated gifs</h1>

## Ch 13

In [44]:
GOES16_samples_13

['/Users/chowdahead/Downloads/GOES_Data/2019/237/OR_ABI-L1b-RadC-M6C13_G16_s20192370101185_e20192370103570_c20192370104093.nc',
 '/Users/chowdahead/Downloads/GOES_Data/2019/237/OR_ABI-L1b-RadC-M6C13_G16_s20192370106185_e20192370108570_c20192370109085.nc',
 '/Users/chowdahead/Downloads/GOES_Data/2019/237/OR_ABI-L1b-RadC-M6C13_G16_s20192370111185_e20192370113570_c20192370114059.nc',
 '/Users/chowdahead/Downloads/GOES_Data/2019/237/OR_ABI-L1b-RadC-M6C13_G16_s20192370116185_e20192370118570_c20192370119087.nc',
 '/Users/chowdahead/Downloads/GOES_Data/2019/237/OR_ABI-L1b-RadC-M6C13_G16_s20192370121185_e20192370123570_c20192370124056.nc',
 '/Users/chowdahead/Downloads/GOES_Data/2019/237/OR_ABI-L1b-RadC-M6C13_G16_s20192370126185_e20192370128570_c20192370129092.nc',
 '/Users/chowdahead/Downloads/GOES_Data/2019/237/OR_ABI-L1b-RadC-M6C13_G16_s20192370131185_e20192370133570_c20192370134058.nc',
 '/Users/chowdahead/Downloads/GOES_Data/2019/237/OR_ABI-L1b-RadC-M6C13_G16_s20192370136185_e201923701385

In [61]:
%%time
print("Here we go...")
channel=13


for i in GOES16_samples_13[:]:
#for i in range(len(GOES16_samples_13[:])):

    #Map(i,channel,save_path,extent,IR_cmap,vmin=162.,vmax=330.)
    Map(i,channel,save_path,extent,my_cmap=IR.IR_Colormap())
    
print("Images done")


Here we go...
Images done
CPU times: user 9min 59s, sys: 33.8 s, total: 10min 32s
Wall time: 9min 12s


In [ ]:
print "gif activated..."

# Set a new list to hold all the images for this channel
files = []
# searching for the files with * 
for name in glob.glob(VISPATH+'*.png'):
    #print name
    files.append(name)

# Make sure our files are in chronological order so our animation is accurate
files = sorted(files, key=lambda x: int(re.sub('\D', '', x)))
    
# Set a new list full of the images so we can create an animation
images = []
for filename in files:
    images.append(imageio.imread(filename))
imageio.mimsave(VISPATH+"VIS_"+IR_path[55:59]+"_"+IR_path[59:62]+".gif", 
                images,duration=0.3)
print "gif done."

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Resize the images if needed

In [ ]:
%cd $save_path

In [ ]:
import glob
from PIL import Image

List = glob.glob("*.png")
for i in List:
    #print(i)
    basewidth = 600
    img = Image.open(i)
    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize), Image.ANTIALIAS)
    img.save('Resized_'+i) 
print(basewidth,hsize)
#img = Image.new('RGB', (basewidth,hsize), (0, 0, 0))
#img.save("GOES_blank_image_Resized.png", "PNG")

In [ ]:
Year = 2019
Julian_Day = 190
# Where the data is supposed to be
GOES_sample_path = '/Users/chowdahead/Downloads/GOES_Data/'+str(Year)+'/'+str(Julian_Day)+'/' # Use full path to file
print("Data path:",GOES_sample_path)
# Check to see if the folder already exists, if not create it
if not os.path.isdir(GOES_sample_path):
    os.makedirs(GOES_sample_path)
    
# Set a path to save the plots with string format for the date to set the month and day
# This will be where the images will be saved
save_path_190 = "/Users/chowdahead/Desktop/Weather_Blog/GOES/"+str(Year)+'/'+str(Julian_Day)+'/'
save_path_190 = save_path_190+"IR/"

print("save path:",save_path_190)

# Check to see if the folder already exists, if not create it
if not os.path.isdir(save_path_190):
    os.makedirs(save_path_190)
    
GOES16_samples_13_190 = []
for i in range(0,24):
    if i < 10:
        hour = "0"+str(i)
    else:
        hour = i
    #print(hour)
    for name in glob.glob(GOES_sample_path+str(hour)+'/*RadC*M6C13*.nc'):
        GOES16_samples_13_190.append(name)
    #for name in glob.glob(GOES_sample_path+'OR*L2*RadC*C09*G16*s'+Year+str(Julian_Day)+'*.nc'):
    #    GOES16_samples_9.append(name)
    GOES16_samples_13_190 = [os.path.join(GOES_sample_path, s) for s in GOES16_samples_13_190]
    GOES16_samples_13_190 = sorted(GOES16_samples_13_190, key=lambda x: int(re.sub('\D', '', x)))

    
print(len(GOES16_samples_13_190))
print(GOES16_samples_13_190[-1])

In [ ]:
Year = 2019
Julian_Day = 189
# Where the data is supposed to be
GOES_sample_path = '/Users/chowdahead/Downloads/GOES_Data/'+str(Year)+'/'+str(Julian_Day)+'/' # Use full path to file
print("Data path:",GOES_sample_path)
# Check to see if the folder already exists, if not create it
if not os.path.isdir(GOES_sample_path):
    os.makedirs(GOES_sample_path)
    
# Set a path to save the plots with string format for the date to set the month and day
# This will be where the images will be saved
save_path_189 = "/Users/chowdahead/Desktop/Weather_Blog/GOES/"+str(Year)+'/'+str(Julian_Day)+'/'
save_path_189 = save_path_189+"IR/"

print("save path:",save_path_189)

# Check to see if the folder already exists, if not create it
if not os.path.isdir(save_path_189):
    os.makedirs(save_path_189)
    
GOES16_samples_13_189 = []
for i in range(0,24):
    if i < 10:
        hour = "0"+str(i)
    else:
        hour = i
    #print(hour)
    for name in glob.glob(GOES_sample_path+str(hour)+'/*RadC*M6C13*.nc'):
        GOES16_samples_13_189.append(name)
    #for name in glob.glob(GOES_sample_path+'OR*L2*RadC*C09*G16*s'+Year+str(Julian_Day)+'*.nc'):
    #    GOES16_samples_9.append(name)
    GOES16_samples_13_189 = [os.path.join(GOES_sample_path, s) for s in GOES16_samples_13_189]
    GOES16_samples_13_189 = sorted(GOES16_samples_13_189, key=lambda x: int(re.sub('\D', '', x)))

    
print(len(GOES16_samples_13_189))
print(GOES16_samples_13_189[-1])

In [ ]:
for i in range(len(GOES16_samples_13_189[:])):

    Map(GOES16_samples_13_189[i],channel,save_path_189,extent,IR_cmap,vmin=162.,vmax=330.)
    Map(GOES16_samples_13_190[i],channel,save_path_190,extent,IR_cmap,vmin=162.,vmax=330.)
    
print("Images done")